In [74]:
import pandas as pd
import numpy as np
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from matplotlib import pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline
import matplotlib
from sklearn import preprocessing
from sklearn import utils
import statsmodels.api as sm
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
matplotlib.rcParams["figure.figsize"] = (20,10)
from sklearn.ensemble import AdaBoostRegressor

In [80]:
df1 = pd.read_csv("training.csv")
# df1.head()

In [81]:
test1 = pd.read_csv("testing.csv")

In [5]:
df1.shape

(100000, 108)

In [6]:
test1.shape

(30000, 107)

In [4]:
from sklearn.linear_model import LinearRegression
clf = LinearRegression()

In [5]:
from sklearn.ensemble import RandomForestRegressor
rfg = RandomForestRegressor()

In [8]:
len(df1.sub_area.unique())

108

# Other data pre processing

In [9]:
df1.sub_area = df1.sub_area.apply(lambda x: x.strip())
sub_area_stats = df1.groupby('sub_area')['sub_area'].agg('count').sort_values(ascending=False)
sub_area_stats

sub_area
Novo-Peredelkino        1203
Bogorodskoe             1169
Vostochnoe Izmajlovo    1158
Lianozovo               1156
Birjulevo Vostochnoe    1152
                        ... 
Prospekt Vernadskogo     263
Krasnosel'skoe           245
Troickij okrug           240
Otradnoe                 108
Obruchevskoe              36
Name: sub_area, Length: 108, dtype: int64

In [10]:
len(sub_area_stats[sub_area_stats<=850])

32

In [11]:
sub_area_stats_less_than_850 = sub_area_stats[sub_area_stats<=850]
sub_area_stats_less_than_850

sub_area
Dorogomilovo                847
Ramenki                     844
Nekrasovka                  810
Cheremushki                 804
Danilovskoe                 797
Severnoe Medvedkovo         796
Izmajlovo                   795
Timirjazevskoe              787
Losinoostrovskoe            779
Akademicheskoe              777
Tekstil'shhiki              760
Donskoe                     754
Filevskij Park              745
Perovo                      729
Jaroslavskoe                690
Vostochnoe Degunino         680
Poselenie Pervomajskoe      648
Taganskoe                   635
Poselenie Filimonkovskoe    627
Vojkovskoe                  577
Beskudnikovskoe             500
Poselenie Sosenskoe         491
Novokosino                  487
Chertanovo Severnoe         473
Kon'kovo                    444
Mar'ina Roshha              418
Begovoe                     373
Prospekt Vernadskogo        263
Krasnosel'skoe              245
Troickij okrug              240
Otradnoe                    108

In [12]:
train2 = df1.copy()
train2.sub_area = train2.sub_area.apply(lambda x: 'other' if x in sub_area_stats_less_than_850 else x)

In [13]:
test1.shape

(30000, 107)

In [14]:
test2 = test1.copy()
test2.sub_area = test2.sub_area.apply(lambda x: 'other' if x in sub_area_stats_less_than_850 else x)

# One Hot Encoding

In [82]:
categorical_cols=["product_type","sub_area","culture_objects_top_25","thermal_power_plant_raion","incineration_raion","oil_chemistry_raion","radiation_raion","railroad_terminal_raion","big_market_raion","nuclear_reactor_raion","detention_facility_raion","water_1line","big_road1_1line","railroad_1line","ecology"]

In [83]:
def onehot_encoding(multi_columns, datafile): 
    df_final = datafile
    i = 0
    for fields in multi_columns:

        print(fields)
        df1 = pd.get_dummies(datafile[fields], drop_first=True)

        datafile.drop([fields], axis=1, inplace=True)
        if i == 0:
            df_final = df1.copy()
        else:

            df_final = pd.concat([df_final, df1], axis=1)
        i = i + 1

    df_final = pd.concat([datafile, df_final], axis=1)

    return df_final


In [84]:
train3 = df1.copy()
train3  = onehot_encoding(categorical_cols, train3)  #applying one hot encoding to the training file

product_type
sub_area
culture_objects_top_25
thermal_power_plant_raion
incineration_raion
oil_chemistry_raion
radiation_raion
railroad_terminal_raion
big_market_raion
nuclear_reactor_raion
detention_facility_raion
water_1line
big_road1_1line
railroad_1line
ecology


In [85]:
test3 = test1.copy()
test3 = onehot_encoding(categorical_cols, test3)

product_type
sub_area
culture_objects_top_25
thermal_power_plant_raion
incineration_raion
oil_chemistry_raion
radiation_raion
railroad_terminal_raion
big_market_raion
nuclear_reactor_raion
detention_facility_raion
water_1line
big_road1_1line
railroad_1line
ecology


In [86]:
Y = train3['price_doc']
train4 = train3.copy()
train4.drop(columns=['ID'], inplace=True) #dropping the row id column as it has no use for training our model
train4.drop(columns=['price_doc'], inplace=True)

# Replacing null values with Mode/Mean

In [87]:
nullCols = ['floor', 'preschool_quota', 'school_quota',
       'raion_build_count_with_material_info', 'build_count_block',
       'build_count_brick', 'build_count_monolith', 'build_count_panel',
       'build_count_foam', 'build_count_before_1920', 'build_count_1946-1970',
       'build_count_1971-1995', 'metro_min_walk', 'railroad_station_walk_km',
       'ID_railroad_station_walk', 'cafe_sum_1000_min_price_avg',
       'cafe_sum_1000_max_price_avg', 'cafe_avg_price_1000',
       'cafe_sum_1500_min_price_avg', 'cafe_sum_1500_max_price_avg',
       'cafe_avg_price_1500', 'cafe_sum_2000_min_price_avg',
       'cafe_sum_2000_max_price_avg', 'cafe_avg_price_2000',
       'cafe_sum_3000_min_price_avg', 'cafe_sum_3000_max_price_avg',
       'cafe_avg_price_3000', 'prom_part_5000', 'cafe_sum_5000_min_price_avg',
       'cafe_sum_5000_max_price_avg', 'cafe_avg_price_5000']

In [88]:
def repWithMode(train,cols):
    for i in cols:
        x = train[i].mode()[0]
        train[i].fillna(x, inplace = True)
    return train        

In [89]:
def repWithMean(train,cols):    
    for i in cols:
        x = train[i].mean()
        train[i].fillna(x, inplace = True)
    return train        

In [90]:
train5 = train4.copy()
train5 = repWithMode(train5,nullCols)
test4 = test3.copy()
test4 = repWithMode(test4,nullCols)

In [60]:
train5 = train4.copy()
train5 = repWithMean(train5,nullCols)
test4 = test3.copy()
test4 = repWithMean(test4,nullCols)

In [91]:
test5 = test4.copy()
test5.drop(columns=['ID'], inplace=True) #dropping the row id column as it has no use for training our model

# Scaling

In [92]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [93]:
sc_X = StandardScaler()
train5 = sc_X.fit_transform(train5)
test5 = sc_X.transform(test5)

In [101]:
MinMax_X = MinMaxScaler()
train5 = MinMax_X.fit_transform(train5)
test5 = MinMax_X.transform(test5)

# Rounding

In [ ]:
train5['floor'] = np.round(train5['floor'])
train5['mosque_count_5000'] = np.round(train5['mosque_count_5000'])

test5['floor'] = np.round(test5['floor'])
test5['mosque_count_5000'] = np.round(test5['mosque_count_5000'])

In [ ]:
train5.drop(["mosque_count_1000"],axis=1,inplace=True)
test5.drop(["mosque_count_1000"],axis=1,inplace=True)

train5.drop(["mosque_count_2000"],axis=1,inplace=True)
test5.drop(["mosque_count_2000"],axis=1,inplace=True)

train5.drop(["mosque_count_3000"],axis=1,inplace=True)
test5.drop(["mosque_count_3000"],axis=1,inplace=True)

# Correlation Filter

In [102]:
from sklearn import datasets

In [94]:
train6 = pd.DataFrame(train5)
cor_matrix = train6.corr().abs()
print(cor_matrix)

          0         1         2         3         4         5         6    \
0    1.000000  0.020234  0.015127  0.039973  0.076501  0.048374  0.070269   
1    0.020234  1.000000  0.007551  0.097418  0.340936  0.817484  0.470750   
2    0.015127  0.007551  1.000000  0.497019  0.247160  0.022571  0.272434   
3    0.039973  0.097418  0.497019  1.000000  0.325450  0.103829  0.331650   
4    0.076501  0.340936  0.247160  0.325450  1.000000  0.225292  0.825862   
..        ...       ...       ...       ...       ...       ...       ...   
210  0.019388  0.039331  0.071137  0.124280  0.132298  0.055020  0.095589   
211  0.021356  0.159894  0.125270  0.139720  0.146653  0.184250  0.086552   
212  0.015641  0.596877  0.286029  0.129887  0.255174  0.567177  0.217346   
213  0.082541  0.140108  0.225218  0.103303  0.243847  0.124619  0.193805   
214  0.070384  0.160520  0.024294  0.060523  0.086715  0.060883  0.005043   

     7         8         9    ...       205       206       207       208  

In [95]:
# Select upper triangle of correlation matrix
upper = cor_matrix.where(np.triu(np.ones(cor_matrix.shape), k=1).astype(np.bool))

C:\Users\hasan\AppData\Local\Temp\ipykernel_17668\1584072055.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = cor_matrix.where(np.triu(np.ones(cor_matrix.shape), k=1).astype(np.bool))


In [96]:
# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
print(to_drop)

[54, 65, 71, 72, 79, 87, 88]


In [108]:
train7 = train6.copy()
# Drop features 
train7.drop(to_drop, axis=1, inplace=True)

In [109]:
test6 = pd.DataFrame(test5)
test6.drop(to_drop, axis=1, inplace=True)

# Adaboost Regressor

In [29]:
DTR=DecisionTreeRegressor(max_depth=1)
RegModel = AdaBoostRegressor(n_estimators=100,base_estimator=DTR ,learning_rate=1)

In [51]:
AB=RegModel.fit(polytrain,Y)

In [52]:
ABans=AB.predict(polytest)

# Gradient Boosting Regressor

In [37]:
GBR = GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=50, random_state=None,
             subsample=1.0, verbose=2, warm_start=False)

In [44]:
train7.shape

(100000, 208)

In [38]:
GBR.fit(polytrain,Y)

C:\Program Files\Python310\lib\site-packages\sklearn\ensemble\_gb.py:294: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  warnings.warn(


      Iter       Train Loss   Remaining Time 
         1 11659312850638.6719            3.37m
         2 10909554310458.5254            3.25m
         3 10551171293414.8945            3.17m
         4 10264775902414.2129            3.10m
         5 10126869199542.5117            3.12m
         6 10017245756007.7559            3.04m
         7 9968217401323.0762            3.09m
         8 9880676494908.0215            3.02m
         9 9812097264862.2930            2.97m
        10 9775471727768.7207            2.94m
        11 9732990271599.9141            2.92m
        12 9680977704539.1387            2.87m
        13 9643838501789.2227            2.80m
        14 9588026175874.1367            2.72m
        15 9527524868897.5781            2.64m
        16 9486176632138.4180            2.56m
        17 9442184670561.2891            2.51m
        18 9409323185998.6680            2.44m
        19 9348266138315.3906            2.36m
        20 9318700049951.5586            2.29m
        

GradientBoostingRegressor(learning_rate=1, loss='ls', n_estimators=50,
                          verbose=2)

In [39]:
GBRans = GBR.predict(polytest)

# Polynomial Regression and Features

In [32]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=3,interaction_only=True)

# Forward Selection

In [22]:
sfs1 = sfs(clf, k_features=10, forward=True, n_jobs =-1,verbose=2, scoring='neg_mean_squared_error')

In [23]:
sfs1 = sfs1.fit(train7, Y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 208 out of 208 | elapsed:   15.8s finished

[2022-11-19 19:52:08] Features: 1/10 -- score: -12861708095086.623[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 207 out of 207 | elapsed:   36.7s finished

[2022-11-19 19:52:44] Features: 2/10 -- score: -12148326675876.402[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 206 out of 206 | elapsed:   39.4s finished

[2022-11-19 19:53:24] Features: 3/10 -- score: -11704713715729.682[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Don

In [24]:
feat_names = list(sfs1.k_feature_names_)
print(feat_names)

[0, 16, 23, 59, 81, 111, 119, 127, 137, 167]


In [25]:
train7 = train7[feat_names]
test6 = test6[feat_names]

# Backward Selection

In [34]:

def backward_elimination(data, target,significance_level = 0.05):
    features = data.columns.tolist()
    while(len(features)>0):
        features_with_constant = sm.add_constant(data[features])
        p_values = sm.OLS(target, features_with_constant).fit().pvalues[1:]
        max_p_value = p_values.max()
        if(max_p_value >= significance_level):
            excluded_feature = p_values.idxmax()
            features.remove(excluded_feature)
        else:
            break 
    return features


In [68]:
f=backward_elimination(train7,Y)
train7=train7[f]
test6 = test6[f]

In [26]:
train7.shape

(100000, 141)

# Decision Tree Regressor

In [45]:
DecRegressor = DecisionTreeRegressor(max_depth=200)

In [46]:
DecRegressor.fit(polytrain,Y)

DecisionTreeRegressor(max_depth=200)

In [47]:
decans = DecRegressor.predict(polytest)

In [54]:
regressor = RandomForestRegressor(n_estimators = 800, random_state = 0,n_jobs = -1,verbose=2)

In [55]:
regressor.fit(polytrain,Y)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 800building tree 2 of 800
building tree 3 of 800

building tree 4 of 800
building tree 5 of 800
building tree 6 of 800
building tree 7 of 800
building tree 8 of 800
building tree 9 of 800
building tree 10 of 800
building tree 11 of 800
building tree 12 of 800
building tree 13 of 800
building tree 14 of 800
building tree 15 of 800
building tree 16 of 800
building tree 17 of 800
building tree 18 of 800
building tree 19 of 800
building tree 20 of 800
building tree 21 of 800
building tree 22 of 800
building tree 23 of 800
building tree 24 of 800
building tree 25 of 800
building tree 26 of 800
building tree 27 of 800
building tree 28 of 800
building tree 29 of 800
building tree 30 of 800
building tree 31 of 800
building tree 32 of 800
building tree 33 of 800
building tree 34 of 800
building tree 35 of 800
building tree 36 of 800
building tree 37 of 800


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  3.2min


building tree 38 of 800
building tree 39 of 800
building tree 40 of 800
building tree 41 of 800
building tree 42 of 800
building tree 43 of 800
building tree 44 of 800
building tree 45 of 800
building tree 46 of 800
building tree 47 of 800
building tree 48 of 800
building tree 49 of 800
building tree 50 of 800
building tree 51 of 800
building tree 52 of 800
building tree 53 of 800
building tree 54 of 800
building tree 55 of 800
building tree 56 of 800
building tree 57 of 800
building tree 58 of 800
building tree 59 of 800
building tree 60 of 800
building tree 61 of 800
building tree 62 of 800
building tree 63 of 800
building tree 64 of 800
building tree 65 of 800
building tree 66 of 800
building tree 67 of 800
building tree 68 of 800
building tree 69 of 800
building tree 70 of 800
building tree 71 of 800
building tree 72 of 800
building tree 73 of 800
building tree 74 of 800
building tree 75 of 800
building tree 76 of 800
building tree 77 of 800
building tree 78 of 800
building tree 79

[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 14.5min


...

building tree 362 of 800
building tree 363 of 800
building tree 364 of 800
building tree 365 of 800
building tree 366 of 800
building tree 367 of 800
building tree 368 of 800
building tree 369 of 800
building tree 370 of 800
building tree 371 of 800
building tree 372 of 800
building tree 373 of 800
building tree 374 of 800
building tree 375 of 800
building tree 376 of 800
building tree 377 of 800
building tree 378 of 800
building tree 379 of 800
building tree 380 of 800
building tree 381 of 800
building tree 382 of 800
building tree 383 of 800
building tree 384 of 800
building tree 385 of 800
building tree 386 of 800
building tree 387 of 800
building tree 388 of 800
building tree 389 of 800
building tree 390 of 800
building tree 391 of 800
building tree 392 of 800
building tree 393 of 800
building tree 394 of 800
building tree 395 of 800
building tree 396 of 800
building tree 397 of 800
building tree 398 of 800
building tree 399 of 800
building tree 400 of 800
building tree 401 of 800


[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 63.1min


building tree 645 of 800
building tree 646 of 800
building tree 647 of 800
building tree 648 of 800
building tree 649 of 800
building tree 650 of 800
building tree 651 of 800
building tree 652 of 800
building tree 653 of 800
building tree 654 of 800
building tree 655 of 800
building tree 656 of 800
building tree 657 of 800
building tree 658 of 800
building tree 659 of 800
building tree 660 of 800
building tree 661 of 800
building tree 662 of 800
building tree 663 of 800
building tree 664 of 800
building tree 665 of 800
building tree 666 of 800
building tree 667 of 800
building tree 668 of 800
building tree 669 of 800
building tree 670 of 800
building tree 671 of 800
building tree 672 of 800
building tree 673 of 800
building tree 674 of 800
building tree 675 of 800
building tree 676 of 800
building tree 677 of 800
building tree 678 of 800
building tree 679 of 800
building tree 680 of 800
building tree 681 of 800
building tree 682 of 800
building tree 683 of 800
building tree 684 of 800


[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed: 78.8min finished


RandomForestRegressor(n_estimators=800, n_jobs=-1, random_state=0, verbose=2)

In [56]:
regans = regressor.predict(polytest)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    3.2s
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed:    8.3s
[Parallel(n_jobs=4)]: Done 640 tasks      | elapsed:   16.7s
[Parallel(n_jobs=4)]: Done 800 out of 800 | elapsed:   20.3s finished


# Linear and Polynomial Regression

In [33]:
polytrain = poly.fit_transform(train7)
polytest = poly.fit_transform(test6)

In [52]:
poly.fit(polytrain, Y)

PolynomialFeatures(degree=5, interaction_only=True)

In [29]:
clf.fit(train7,Y)

LinearRegression()

In [30]:
ans = clf.predict(test6)

# Extra Tree Regressor

In [110]:
extra_tree_model = ExtraTreesRegressor(n_estimators = 1500,verbose=2,criterion ='mse', max_features = 'auto')

In [111]:
extra_tree_model.fit(train7, Y)

C:\Program Files\Python310\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Program Files\Python310\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 800building tree 2 of 800

building tree 3 of 800
building tree 4 of 800
building tree 5 of 800
building tree 6 of 800
building tree 7 of 800
building tree 8 of 800
building tree 9 of 800
building tree 10 of 800
building tree 11 of 800
building tree 12 of 800
building tree 13 of 800
building tree 14 of 800
building tree 15 of 800
building tree 16 of 800
building tree 17 of 800
building tree 18 of 800
building tree 19 of 800
building tree 20 of 800
building tree 21 of 800
building tree 22 of 800
building tree 23 of 800
building tree 24 of 800
building tree 25 of 800
building tree 26 of 800
building tree 27 of 800
building tree 28 of 800
building tree 29 of 800
building tree 30 of 800
building tree 31 of 800
building tree 32 of 800
building tree 33 of 800
building tree 34 of 800
building tree 35 of 800
building tree 36 of 800
building tree 37 of 800


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.2min


building tree 38 of 800
building tree 39 of 800
building tree 40 of 800
building tree 41 of 800
building tree 42 of 800
building tree 43 of 800
building tree 44 of 800
building tree 45 of 800
building tree 46 of 800
building tree 47 of 800
building tree 48 of 800
building tree 49 of 800
building tree 50 of 800
building tree 51 of 800
building tree 52 of 800
building tree 53 of 800
building tree 54 of 800
building tree 55 of 800
building tree 56 of 800
building tree 57 of 800
building tree 58 of 800
building tree 59 of 800
building tree 60 of 800
building tree 61 of 800
building tree 62 of 800
building tree 63 of 800
building tree 64 of 800
building tree 65 of 800
building tree 66 of 800
building tree 67 of 800
building tree 68 of 800
building tree 69 of 800
building tree 70 of 800
building tree 71 of 800
building tree 72 of 800
building tree 73 of 800
building tree 74 of 800
building tree 75 of 800
building tree 76 of 800
building tree 77 of 800
building tree 78 of 800
building tree 79

[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  5.1min


...

building tree 362 of 800
building tree 363 of 800
building tree 364 of 800
building tree 365 of 800
building tree 366 of 800
building tree 367 of 800
building tree 368 of 800
building tree 369 of 800
building tree 370 of 800
building tree 371 of 800
building tree 372 of 800
building tree 373 of 800
building tree 374 of 800
building tree 375 of 800
building tree 376 of 800
building tree 377 of 800
building tree 378 of 800
building tree 379 of 800
building tree 380 of 800
building tree 381 of 800
building tree 382 of 800
building tree 383 of 800
building tree 384 of 800
building tree 385 of 800
building tree 386 of 800
building tree 387 of 800
building tree 388 of 800
building tree 389 of 800
building tree 390 of 800
building tree 391 of 800
building tree 392 of 800
building tree 393 of 800
building tree 394 of 800
building tree 395 of 800
building tree 396 of 800
building tree 397 of 800
building tree 398 of 800
building tree 399 of 800
building tree 400 of 800
building tree 401 of 800


[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 22.0min


building tree 645 of 800
building tree 646 of 800
building tree 647 of 800
building tree 648 of 800
building tree 649 of 800
building tree 650 of 800
building tree 651 of 800
building tree 652 of 800
building tree 653 of 800
building tree 654 of 800
building tree 655 of 800
building tree 656 of 800
building tree 657 of 800
building tree 658 of 800
building tree 659 of 800
building tree 660 of 800
building tree 661 of 800
building tree 662 of 800
building tree 663 of 800
building tree 664 of 800
building tree 665 of 800
building tree 666 of 800
building tree 667 of 800
building tree 668 of 800
building tree 669 of 800
building tree 670 of 800
building tree 671 of 800
building tree 672 of 800
building tree 673 of 800
building tree 674 of 800
building tree 675 of 800
building tree 676 of 800
building tree 677 of 800
building tree 678 of 800
building tree 679 of 800
building tree 680 of 800
building tree 681 of 800
building tree 682 of 800
building tree 683 of 800
building tree 684 of 800


[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed: 25.1min finished


ExtraTreesRegressor(criterion='mse', max_features='auto', n_estimators=800,
                    n_jobs=-1, verbose=2)

In [112]:
etm = extra_tree_model.predict(test6)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    1.2s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    6.4s
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed:   14.2s
[Parallel(n_jobs=4)]: Done 640 tasks      | elapsed:   26.0s
[Parallel(n_jobs=4)]: Done 800 out of 800 | elapsed:   31.5s finished


In [113]:
output = pd.read_csv('sample.csv')
output['price_doc'] = etm
output.to_csv('output50.csv',index=False)